In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Imports**
---



In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp
from datetime import date


pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
p = Path('C:/Users/alexd/Dropbox/FFE')

edge_file = p / 'FinnShapeEdges.parquet'
wind_file = p / 'Copy of GD_wind.csv'
folder = Path('F:/output')

In [4]:
# load dataaaa
wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [5]:
rngFile = edgelist[['source', 'IgnProbBld']]
rngFile.drop_duplicates(['source'], inplace=True)

In [6]:
rngFile.IgnProbBld.describe()
rngFile.source.describe()

count    69824.000000
mean         0.000129
std          0.000223
min          0.000024
25%          0.000061
50%          0.000092
75%          0.000124
max          0.005239
Name: IgnProbBld, dtype: float64

count    69824.000000
mean     36867.715742
std      21298.611168
min          0.000000
25%      18424.750000
50%      36860.500000
75%      55315.250000
max      73734.000000
Name: source, dtype: float64

**Definitions & parallel computing**
---

In [7]:
# num_cores = multiprocessing.cpu_count()
# print(num_cores)
# # need to do this on Shell / Terminal:
# conda install ipyparallel
# ipcluster nbextension enable --user
# ipcluster start # or ipcluster start -n 4

In [2]:
import ipyparallel as ipp
client = ipp.Client()
dview = client[:]
client.ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79]

In [9]:
# add variables to all engines
dview["edgelist"]=edgelist
dview["rngFile"]=rngFile
dview["wind_data"]=wind_data
dview["folder"]=folder

# add all libraries to engines
with dview.sync_imports():
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import sys
    import os
    import glob
    from datetime import date

importing numpy on engine(s)
importing pandas on engine(s)
importing Path from pathlib on engine(s)
importing sys on engine(s)
importing os on engine(s)
importing glob on engine(s)
importing date from datetime on engine(s)


In [10]:
%%px

def wind_scenario(wind_data):
      import numpy as np
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    import numpy as np
    import pandas as pd
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProbBld']]
    initialIgnitions = len(rngList)
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges, initialIgnitions


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


In [11]:
@dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    from datetime import date
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges, numberIgnitions = ignition()
        if ActiveEdges.empty:
            print(f"no ignitions {numberIgnitions}")
            condition = False
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            burns = ActiveEdges.drop_duplicates(['source'], inplace=False)
#             print(f"Active edges {len(ActiveEdges)} / no duplicate = {len(burns)}")
            burns['time'] = time
            listScenarioDataframes.append(burns)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing scenario --- {scenario} time ---- {time} \n started with {numberIgnitions} ignitions ')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["InitialIgnitions"] = numberIgnitions
        Activations.to_parquet(str(folder) + '/' + f'scenario{scenario}_{str(date.today())}.parquet', engine='auto', compression="GZIP")
        

**Main**
---


---



In [12]:
%%time
ffe_runs(range(3000))

Wall time: 6 ms


<AsyncMapResult: ffe_runs>


**Backup**
---



---



In [16]:
test = pd.read_parquet(folder / 'scenario0_2020-08-06.parquet')
test

,source,target,distance,bearing,IgnProbBld,time,scenario,InitialIgnitions
58927,1339,7165,85.191863,254.643103,0.000073,0,0,8
177754,4087,1171,42.839607,146.200091,0.000068,0,0,8
1553230,36012,20709,46.135131,327.995628,0.005239,0,0,8
1838643,42567,819,61.723592,176.924230,0.000288,0,0,8
1870466,43322,69,18.925020,224.543699,0.000253,0,0,8
2437404,56553,2724,79.663636,261.571453,0.000117,0,0,8
2500477,58033,38,73.423960,28.218908,0.000313,0,0,8
3072849,71356,513,65.566426,303.012553,0.000213,0,0,8


In [14]:
# def clean_up(path=folder):
#     files = path.glob('*scenario*')
#     for f in files:
#         print(f)
#         os.remove(f)

In [15]:
# clean_up()